In [9]:
!pip install overrides
!pip install tqdm
!pip install efficientnet_pytorch


In [ ]:
!wget https://dwz.cn/ijPVPQhz
!unrar x ijPVPQhz

In [21]:
# 头文件
from overrides import overrides
import torch.nn as nn
import torch
import pandas as pd
from torchvision.datasets.folder import accimage_loader, pil_loader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import lr_scheduler
import os
import torchvision
from torchvision import transforms
from typing import Tuple
from torch.nn import Module
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

In [24]:
class Trainer:
    """Trainer class to abstract rudimentary training loop."""

    def __init__(
            self,
            model: Module,
            criterion: Module,
            optimizer: Optimizer,
            device: torch.device) -> None:
        """Set trainer class with model, criterion, optimizer. (Data is passed to train/eval)."""
        super(Trainer, self).__init__()
        self.model: Module = model
        self.criterion: Module = criterion
        self.optimizer: Optimizer = optimizer
        self.device: torch.device = device

    def train(self, loader: DataLoader) -> Tuple[float, float]:
        """Train model using batches from loader and return accuracy and loss."""
        total_loss, total_acc = 0.0, 0.0
        self.model.train()
        try:
            with tqdm(enumerate(loader), total=len(loader), desc='Training') as proc:
                for _, (inputs, targets) in proc:
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, targets)
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
                    _, predicted = torch.max(outputs, 1)
                    total_loss += loss.item()
                    total_acc += (predicted == targets).float().sum().item() / targets.numel()
        except Exception as e:
            # 异常情况关闭
            print("Running Error in training, ", e)
            proc.close()
            return -1, -1
        proc.close()
        return total_loss / len(loader), 100.0 * total_acc / len(loader)

    def test(self, loader: DataLoader) -> Tuple[float, float]:
        """Evaluate model using batches from loader and return accuracy and loss."""
        with torch.no_grad():
            total_loss, total_acc = 0.0, 0.0
            self.model.eval()
            try:
                with tqdm(enumerate(loader), total=len(loader), desc='Testing ') as proc:
                    for _, (inputs, targets) in proc:
                        inputs = inputs.to(self.device)
                        targets = targets.to(self.device)
                        outputs = self.model(inputs)
                        loss = self.criterion(outputs, targets)
                        _, predicted = torch.max(outputs, 1)
                        total_loss += loss.item()
                        total_acc += (predicted == targets).float().sum().item() / targets.numel()
            except Exception as e:
                proc.close()
                print("Running Error in validating,", e)
                return -1, -1
            proc.close()
        return total_loss / len(loader), 100.0 * total_acc / len(loader)

    def predict(self, loader: DataLoader):
        results = []
        for inputs, targets in loader:
            inputs = inputs.to(self.device)
            outputs = self.model(inputs)
            _, predicted = torch.max(outputs, 1)
            results.append(predicted)
        return results


def run_epochs_for_loop(
        trainer: Trainer,
        epochs: int,
        train_loader: DataLoader,
        test_loader: DataLoader,
        scheduler: ReduceLROnPlateau = None):
    # Run train + evaluation loop for specified epochs.
    global best 
    for epoch in range(epochs):
        (train_loss, train_acc) = trainer.train(train_loader)
        (test_loss, test_acc) = trainer.test(test_loader)
        print()
        print("Epoch %d: TrainLoss %f \t TrainAcc %f" % (epoch+1, train_loss, train_acc))
        print("Epoch %d: TestLoss %f \t TestAcc %f" % (epoch+1, test_loss, test_acc))
        # 动态更新学习率
        if scheduler is not None:
            scheduler.step()
        # 保存训练的结果
        if test_acc > best:
            best = test_acc
            save_checkpoint(trainer, epoch, test_acc, "./")


def save_checkpoint(trainer: Trainer, epoch: int, accuracy: float, path: str):
    # 保存训练结果
    path = os.path.join(path, "checkpoint.pt")
    checkpoint = {
        "model": trainer.model.state_dict(),
        "optimizer": trainer.optimizer.state_dict(),
        "epoch": epoch,
        "accuracy": accuracy,
    }
    torch.save(checkpoint, path)


def save_model(model, path):
    # 只保存模型的实例变量
    if torch.__version__ >= "1.6.0":
        torch.save(model.state_dict(), path, _use_new_zipfile_serialization=False)
    else:
        torch.save(model.state_dict(), path)

In [12]:


def conv3x3(in_planes, out_planes, stride=1):
    # "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # 压缩空间
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out  # [b, C, 1, 1]
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)  # 压缩通道
        max_out, _ = torch.max(x, dim=1, keepdim=True)   # 压缩通道
        x = torch.cat([avg_out, max_out], dim=1)  # [b, 1, h, w]
        x = self.conv1(x)
        return self.sigmoid(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)

        self.ca = ChannelAttention(planes * 4)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [13]:
class EfficientNetWithAttention(nn.Module):

    def __init__(self, num_classes: int = 200):
        super(EfficientNetWithAttention, self).__init__()
        self.eff_model = EfficientNet.from_pretrained("efficientnet-b5")
        self._avg_pooling = nn.AdaptiveAvgPool2d(output_size=1)
        self._dropout = nn.Dropout(p=0.5, inplace=False)

        self._fc = nn.Linear(in_features=self.eff_model._fc.in_features, out_features=num_classes, bias=True)
        self.ca_head = ChannelAttention(64)
        self.sa = SpatialAttention()
        self.ca_tail = ChannelAttention(self.eff_model._fc.in_features)

    def forward(self, x):
        x = self.eff_model.extract_features(x)
        x = self.ca_tail(x) * x
        x = self.sa(x) * x

        x = self._avg_pooling(x)
        if self.eff_model._global_params.include_top:
            x = x.flatten(start_dim=1)
            x = self._dropout(x)
            x = self._fc(x)
        return x

In [14]:

data_dir = "./AI研习社_鸟类识别比赛数据集"
selection = ["train_set", "val_set"]
train_labels = os.path.join(data_dir, "train_pname_to_index.csv")
valid_labels = os.path.join(data_dir, "val_pname_to_index.csv")


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)


class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, data_label_path, data_transform, data_loader=default_loader):
        """

        :param data_path: 要读取的文件的路径
        :param data_label_path: 标签数据的路径
        :param data_transform: 数据变换模式
        :param data_loader: 加载方法
        """
        # 在label文件中注意不要加上标签
        df = pd.read_csv(data_label_path, header=None)
        self.data_loader = data_loader
        self.data_transform = data_transform
        self.data_path = data_path
        # 获取文件夹下的全部图片名
        self.img_names = list(df[0])
        self.labels = list(df[1])

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, item):
        img_name = self.img_names[item]
        img_path = os.path.join(self.data_path, img_name)
        label = self.labels[item]
        img = self.data_loader(img_path)
        try:
            img = self.data_transform(img)
            return img, label-1
        except:
            raise Exception("cannot transform image: {}".format(img_name))


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    formats = {
        'train_set': [
            transforms.Resize(456),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(5),
            transforms.RandomCrop(456),
            transforms.ToTensor(),
            normalize,
        ],
        'val_set': [
            transforms.Resize(456),
            transforms.CenterCrop(456),
            transforms.ToTensor(),
            normalize,
        ],
    }
    data_label_paths = {
        "train_set": train_labels,
        "val_set": valid_labels,
    }
    data_sets = {}
    for one in selection:
        data_sets[one] = CustomDataset(os.path.join(data_dir, one),
                                       data_label_paths[one],
                                       transforms.Compose(formats[one]))
    data_loader = {
        "train":
            torch.utils.data.DataLoader(
                data_sets["train_set"],
                batch_size=64,
                shuffle=True,
                num_workers=4
            ),

        "valid": torch.utils.data.DataLoader(
                data_sets["val_set"],
                batch_size=5,
                shuffle=False,
                num_workers=4
            ),
    }
    # eff_model = EfficientNet.from_pretrained("efficientnet-b5")
    # in_features = eff_model._fc.in_features
    # eff_model._fc = nn.Linear(in_features, 200)
    eff_model = EfficientNetWithAttention()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5


In [31]:
data_loader = {
        "train":
            torch.utils.data.DataLoader(
                data_sets["train_set"],
                batch_size=64,
                shuffle=True,
                num_workers=4
            ),

        "valid": torch.utils.data.DataLoader(
                data_sets["val_set"],
                batch_size=5,
                shuffle=False,
                num_workers=4
            ),
    }
for param in eff_model.parameters():
    param.requires_grad = False
for param in eff_model._fc.parameters():
    param.requires_grad = True
for param in eff_model.ca_tail.parameters():
    param.requires_grad = True
for param in eff_model.sa.parameters():
    param.requires_grad = True

best = 80

# Observe that all parameters are being optimized
params = filter(lambda p: p.requires_grad, eff_model.parameters())
optimizer = torch.optim.SGD(params, lr=0.003, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion = nn.CrossEntropyLoss()
eff_model.to(device)
criterion.to(device)
trainer = Trainer(eff_model, criterion, optimizer, device)
run_epochs_for_loop(trainer, 21, data_loader["train"], data_loader["valid"], exp_lr_scheduler)

Testing : 100%|██████████| 354/354 [00:55<00:00,  6.35it/s]



Epoch 1: TrainLoss 0.472730 	 TrainAcc 88.753531
Epoch 1: TestLoss 1.033943 	 TestAcc 81.487759


Testing : 100%|██████████| 354/354 [00:55<00:00,  6.34it/s]



Epoch 2: TrainLoss 0.460492 	 TrainAcc 88.884715
Epoch 2: TestLoss 1.030026 	 TestAcc 81.544256


Testing : 100%|██████████| 354/354 [00:55<00:00,  6.35it/s]

...


Epoch 19: TrainLoss 0.453402 	 TrainAcc 88.805060
Epoch 19: TestLoss 1.023744 	 TestAcc 81.261770



Testing : 100%|██████████| 354/354 [00:55<00:00,  6.33it/s]



Epoch 20: TrainLoss 0.415365 	 TrainAcc 89.441885
Epoch 20: TestLoss 1.021756 	 TestAcc 81.600753


Testing : 100%|██████████| 354/354 [00:55<00:00,  6.34it/s]


Epoch 21: TrainLoss 0.454672 	 TrainAcc 88.886768
Epoch 21: TestLoss 1.020555 	 TestAcc 81.487759


In [32]:
checkpoint = torch.load("checkpoint.pt")
eff_model.load_state_dict(checkpoint['model'])
acc = checkpoint["accuracy"]
print("Load Ok, accuracy=",acc)

Load Ok, accuracy= 81.60075329566875


In [8]:
!unzip checkpoint_best.zip

Archive:  checkpoint_best.zip
  inflating: checkpoint_best.pt      


In [ ]:
data_loader = {
        "train":
            torch.utils.data.DataLoader(
                data_sets["train_set"],
                batch_size=8,
                shuffle=True,
                num_workers=4
            ),

        "valid": torch.utils.data.DataLoader(
                data_sets["val_set"],
                batch_size=5,
                shuffle=False,
                num_workers=4
            ),
    }
# 微调
for param in eff_model.parameters():
    param.requires_grad = True
lr = 0.001
optimizer = torch.optim.Adam(eff_model.parameters(), lr=lr)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
trainer = Trainer(eff_model, criterion, optimizer, device)
run_epochs_for_loop(trainer, 22, data_loader["train"], data_loader["valid"], exp_lr_scheduler)

In [ ]:
eff_model = EfficientNet.from_pretrained("efficientnet-b7")
print(eff_model.__dict__)

In [30]:
# 修改test_set为test，增加一个test文件夹嵌套
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
trans = transforms.Compose([
    transforms.Resize(456),
    transforms.CenterCrop(456),
    transforms.ToTensor(),
    normalize,
])
predict_sets = torchvision.datasets.ImageFolder(os.path.join(data_dir, "test"), transform=trans)
test_loader = torch.utils.data.DataLoader(
            predict_sets,
            batch_size=1, # 一次一张一张的预测
            shuffle=False,
            num_workers=4
        )
tt = Trainer(eff_model, criterion, optimizer, device)
predictions = tt.predict(test_loader)
answer = []
for index, cls in enumerate(predictions):
      # print(predictions[index])
      # break
      path = predict_sets.imgs[index][0]
      l = path.split("/")
      img_name = l[-1]
      answer.append((img_name, int(predictions[index])+1))
answer = sorted(answer, key=lambda x: int(x[0].split(".")[0]))
import csv
with open('test.csv','w', newline="")as f:
    writer = csv.writer(f)
    for one in answer:

      writer.writerow([one[0], one[1]])